# Latent consistency model using Optimum-Intel OpenVINO and multiple Intel Hardware



In [ ]:
%pip install -q "optimum-intel[openvino,diffusers]@git+https://github.com/huggingface/optimum-intel.git" "ipywidgets" "transformers>=4.33.0" --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Showing Info Available Devices
[back to top ⬆️](#Table-of-contents:)

The `available_devices` property shows the available devices in your system. The "FULL_DEVICE_NAME" option to `ie.get_property()` shows the name of the device. Check what is the ID name for the discrete GPU, if you have integrated GPU (iGPU) and discrete GPU (dGPU), it will show `device_name="GPU.0"` for iGPU and `device_name="GPU.1"` for dGPU. If you just have either an iGPU or dGPU that will be assigned to `"GPU"`

Note: For more details about GPU with OpenVINO visit this [link](https://docs.openvino.ai/nightly/openvino_docs_install_guides_configurations_for_intel_gpu.html). If you have been facing any issue in Ubuntu 20.04 or Windows 11 read this [blog](https://blog.openvino.ai/blog-posts/install-gpu-drivers-windows-ubuntu).

In [ ]:
from openvino.runtime import Core

ie = Core()
devices = ie.available_devices

for device in devices:
    device_name = ie.get_property(device, "FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

### Using full precision model in CPU with `DiffusionPipeline`
[back to top ⬆️](#Table-of-contents:)


In [ ]:
from diffusers import LatentConsistencyModelPipeline
import gc

pipeline = LatentConsistencyModelPipeline.from_pretrained("SimianLuo/LCM_Dreamshaper_v7")
prompt = "sailing ship in storm by Leonardo da Vinci"

image = pipeline(
    prompt=prompt, num_inference_steps=4, guidance_scale=8.0
).images[0]
image.save("image_cpu.png")
image

In [ ]:
del pipeline
gc.collect()

### Using full precision model in CPU with `OVLatentConsistencyModelPipeline.`
[back to top ⬆️](#Table-of-contents:)


In [ ]:
from optimum.intel import OVLatentConsistencyModelPipeline

ov_pipeline= OVLatentConsistencyModelPipeline.from_pretrained("SimianLuo/LCM_Dreamshaper_v7", export=True)


In [ ]:
prompt = "sailing ship in storm by Leonardo da Vinci"
image_ov_cpu = ov_pipeline(prompt=prompt, num_inference_steps=4, guidance_scale=8.0).images[0]
image_ov_cpu.save("image_opt_cpu.png")
image_ov_cpu

### Using full precision model in dGPU with `OVLatentConsistencyModelPipeline`
[back to top ⬆️](#Table-of-contents:)

The model in this notebook is FP32 precision. And thanks to the new feature of OpenVINO 2023.0 you do not need to convert the model to FP16 for running the inference on GPU.


In [ ]:
ov_pipeline.to("GPU")
ov_pipeline.compile()

In [ ]:
image_ov_gpu = ov_pipeline(prompt=prompt, num_inference_steps=4, guidance_scale=8.0).images[0]
image_ov_gpu.save("image_opt_cpu.png")
image_ov_gpu

In [ ]:
del ov_pipeline
gc.collect()